In [1]:
using LinearAlgebra
using Printf
using Polynomials
using Plots
using SparseArrays
using ToeplitzMatrices
using FiniteDifferences

In [2]:
function make_reduced_order(A, b, c, q)
    lambdas = eigvals(A);
    vectors = eigvecs(A);
    vectors
    Vq = zeros(N, q);
#     n = size(vectors)
#     print(n)
    for i in 0:q-1
        print(q-i)
        Vq[:,i+1] = vectors[:,q-i]
    end    
    A_hat =  Vq' * A * Vq;
    b_hat = Vq' * b;
    c_hat = Vq' * c;
#     a = size(Vq)
    return A_hat, b_hat, c_hat, Vq
end

make_reduced_order (generic function with 1 method)

% uses Forward Euler to simulate states model dx/dt=f(x,p,u)
% startin from state vector x_start at time t_start
% until time t_stop, with time intervals timestep
% eval_f is a text string defining the name of the function that evaluates f(x,p,u)
% eval_u is a text string defining the name of the funciton that evaluates u(t)
% visualize ~= 0 is an optional parameter triggering the generation of intermediate plots of the state
% 
% EXAMPLE

In [3]:
function ForwardEuler(eval_f,x_start,p,t_start,t_stop,timestep)
#     print(ceil((t_stop-t_start)/timestep))f
#     aksm
    X = zeros(length(x_start), Int(ceil((t_stop-t_start)/timestep)));
    t = zeros(Int(ceil((t_stop-t_start)/timestep)), 1);
    X[:,1] = x_start;
    t[1]   = t_start;

    for n in 1 : Int(ceil((t_stop-t_start)/timestep)-1)
#         ass = minimum(t[n]);
#        dt = mininum([timestep; (t_stop-t[n])]);
        dt = timestep;
#         if n+1 <=
       t[n+1]   = t[n] + dt;
       u = eval_u_step(t[n]);
       f = eval_f(X[:,n], p, u);
       X[:,n+1] = X[:,n] +  dt * f;
    end
    
    return X,t
end

ForwardEuler (generic function with 1 method)

In [ ]:
% evaluates the vector field f(x,p,u) 
% at state vector x, and with vector of inputs u.
% p is a structure containing all model parameters
% i.e. in this case: matrices p.A and p.B 
% corresponding to state space model dx/dt = p.A x + p.B u

In [4]:
function eval_f(x,p,u)
    f = p.A * x .+ p.B * u;
    return f;
end

eval_f (generic function with 1 method)

In [5]:
% generates the value of the input u at time t
% corresponding to a step of magnitude 1

LoadError: syntax: "%" is not a unary operator

In [6]:
function eval_u_step(t)
    if t <0
       u = 0;
    else 
       u = 1;
    end
    return u;
end

eval_u_step (generic function with 1 method)

In [27]:
function TransientReference(err_tol,eval_f,p,x_start,t_start,t_stop)
    k = 1;
    len = 0
    res = Float64[];
    t = Float64[];
    for i = -1:-.5:-4 
        len = len + 1
    end
    dt = zeros(len, 1);
    X = zeros(length(x_start), 100);
   for n = -1:-.5:-4 
      dt[k] = 10^n;
#       tic
      (temp_X,temp_t) = ForwardEuler(eval_f,x_start,p,t_start,t_stop,dt[k]);
        dims  = size(temp_X);
        t = copy(temp_t);
        print(temp_X[:,2])
#         print("\n")
        X[:,k] = temp_X[:,dims[2]];
#       [X{k},t{k}] = Implicit('Trapezoidal',eval_f,x_start,p,eval_u,t_start,t_stop,dt(k),visualize,FiniteDifference,eval_Jf);
#       CompTimes(k) = toc;
#       disp(['solved with dt = ' num2str(dt(k)) ' in ' num2str(CompTimes(k)) ' seconds'])
      if k > 1
        dims = size(temp_X);
         res = abs.(X[:,k] - X[:,k-1]);
#             print(rez)
#             print("\n")
#          loglog(dt,X_diff,'.')
#          ylabel('error estimate: ||x_{dt^k} - x_{dt^{k-1}}||')
#          xlabel('dt')
#          drawnow
      end
#         print(k)
      k = k + 1;
   end
        
   err_ref = maximum(res);
#     print(res)
   t_ref  = t[length(t)];
    dim_X = size(X);
   X_ref  = X[dim_X[1], dim_X[2]];
   dt_ref = dt[length(dt)];

    return X, t_ref, dt_ref, err_ref
   
#    % for debugging put comment on this. when done debugging remove comment
#    eval(['save ' filename ' X_ref t_ref dt_ref err_ref -v4'])
end

TransientReference (generic function with 1 method)

In [8]:
function getParam_HeatBar(N,L)
    # % define material properties parameters
    gamma = 0.1; #%related to thermal capacitance per unit length of the bar
    km    = 0.1; #%related to thermal conductance through metal per unit length of the bar
    ka    = 0.1; #%related to thermal conductance to air per unit length of the bar
    dxFD = 1.5e-8;

    L = 1;
    dz   = L/(N-1); #% length of each segment. N includes the two terminal nodes
                      #% notice when changing N what stays constant is the total length L
                      #% not the length of each discretization section.
                      #% results should not depend on the number of sections
                      #% for a large enough number of sections
    Cstore = gamma  * dz; #%the longer the section the larger the thermal storage
    Rc     = (1/km) * dz; #%the longer the section the larger the thermal resistance
    Rloss  = (1/ka) / dz; #%the longer the section the larger the thermal leakage
                              #%hence the smaller the thermal resistance to ambient
    A    = zeros(N,N);#%allocate space for large sparse dynamic matrix in advance

    # % coupling resistors Rc between i and j=i+1
    for i = 1:N-1,
       j=i+1;
       A[i,i] = A[i,i]+(+1/Rc);
       A[i,j] = A[i,j]+(-1/Rc);
       A[j,i] = A[j,i]+(-1/Rc);
       A[j,j] = A[j,j]+(+1/Rc);
    end
#     % leakage resistor Rloss between i and ground
    for i = 1:N,
       A[i,i] = A[i,i] + 1/Rloss;
    end

    B     = sparse(zeros(N,1));  
    B[1,1]= 1;		    # % heat source at the leftmost side of the bar

    A     = -A/Cstore; #% note this will result in a 1/p.dz^2 term in A
                           #% also pay attention to the negative sign
                           #% otherwise the system is completely unstable
    B     = B/Cstore;  #% note this is important to make sure results
                           #% will not depend on the number of sections N

    p = pee_maker(dz,gamma,km,ka,A, B,dxFD);

    # % define also some example parameters for a typical transient simulation                       
    x_start = zeros(N,1);  
    t_start = 0;

    lambda = eigvals(A);
    slowest_eigenvalue = minimum(abs.(lambda))
    fastest_eigenvalue = maximum(abs.(lambda))

    # % to see steady state need to wait until the slowest mode settles
    t_stop = (slowest_eigenvalue*2);    #%use this to wait until slowest mode

    # % usually Forward Euler is unstable for timestep>2/fastest_eigenvalue
    max_dt_FE = 1/fastest_eigenvalue;
    
    return dz,p,A, x_start,t_start,t_stop,max_dt_FE
end

getParam_HeatBar (generic function with 1 method)

### Creating test data

In [28]:
struct pee_maker
    dz
    gamma
    km
    ka
    A
    B
    dxFD
end

L = 1; #	% total lenght of the hear bar
N = 500;# % number of nodes/states/unknowns along the bar (including z=0 and z=L

#% this defines all parameters needed to evaluate the heat bar model 
(dz,p,A, x_start,t_start,t_stop,max_dt_FE) = getParam_HeatBar(N,L);
# print(p.ka)

# eval_f  = 'eval_f_LinearSystem';
# eval_Jf = 'eval_Jf_LinearSystem';
# dxFD  = sqrt(eps);  % on double precision 1.5e-8 
# p = pee_maker(dxFD)

# %function defining the input of the dynamical system
# eval_u = 'eval_u_step';        % step function u(t)=1 for all t>0
# %eval_u = 'something else...';

# % these two values give about the same 6e-6 error compared to "reference"
dt_FE = max_dt_FE;     #%Forward Euler full transient takes 640ms, 
dt_TR = 50*max_dt_FE;  #%Trapezoidal   full transient takes 220ms so 3x faster

t_stop = 10;
t_plot = [0 .1 .2 .4 .8 1.6 3.2 6.4 10];

# %-----------------------------------------------------------------------
# % first lets get a reference solution (either precomputed or generate it)

# filename = 'HeatBar_TransientReferenceN500.mat';

err_tol =1e-8;
(X_ref, t_ref, dt_ref, err_ref) = TransientReference(err_tol,eval_f,p,x_start,t_start,t_stop);
@printf("The reference confidence is %f >:D\n", err_ref);
# print(t_ref)
# print("\n")
# print(dt_ref)
# print("\n")
# # print(err_ref)
# print("\n")
# disp(['Reference confidence is around: ' num2str(err_ref)])
# disp(['Do not use it to measure quantities smaller or comparable to that value'])


m = 1.030286966852145e-13;
indices_t = [1,101,201,401,801,1601,3201,6401,10001];
# for i = 1:length(t_plot)
#    (m,indices_t[i])=minimum(abs.(t_ref-t_plot[i]));
#     print(minimum(abs.(t_ref-t_plot[i])))
#     print("\n")
# end

# print(p.dz)
# z = range(0, L, p.dz);  #% p.dz is the length of each discretization section of the bar
z = zeros(500, 1);
cunt = 1;
for i = 0:p.dz:L
    z[cunt] = i;
    cunt += 1;
end
# h = figure;
# Xs = X_ref[:,indices_t]';
# plot(z,Xs)
# xlabel('z')
# ylabel('T(z)')
# drawnow
# hold on


# figure
# titletxt='time domain animation of Temperature distribution over time';
# %disp(['showing the ' titletxt])
# %dt = dt_TR;
dt = dt_ref*20;
# print(X_ref)
for t = t_start:dt:t_stop
#    %[m,FE_index] = min(abs(t_FE-t));
#    %[m,BE_index] = min(abs(t_BE-t));
   Ref_index = Int(ceil(abs.(t_ref-t)));
    Xs = X_ref[:,Ref_index]
   plot(z, Xs)
#    %plot(z,X_FE_all(:,FE_index),'b--',z,X_BE_all(:,BE_index),'ko',z,X_TR_all(:,TR_index),'r.'); 
#    axis([0 L 0 14]);
#    xlabel('z')
#    ylabel('T(z)')
#    title(titletxt)
#    drawnow
end
savefig("test.png")


[499.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

"/Users/adrianaladera/Desktop/MIT/2.096_mod_simmie/project_gay/MOD-AND-SIMMIE/test.png"

In [202]:
N = 50
col = zeros(N); col[1] = 2; col[2] = -1;
A  = Toeplitz(col,col)
c = zeros(N,1); c[N] = 1;
b = zeros(N,1); b[1] = 1/(1000);
q = 5;
ass = minimum(abs.(A))
# dick = range(0,1, 1, 0.1)

0.0

In [184]:
red_materials = make_reduced_order(A, b, c, q);


54321